In [ ]:
# _**Import Package**_

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

# _**Load Dataset**_

iris_dataset = load_iris()

iris_dataset

data = iris_dataset['data']
target = iris_dataset['target']

# _**Train_Test_Split Data**_

x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

# _**Data Loader**_

class IRISDataset():
  def __init__(self, x, y):
    self.x = torch.tensor(x, dtype=torch.float32)
    self.y = torch.tensor(y)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]    

iris_dataset_train = IRISDataset(x_train, y_train)
iris_dataset_test = IRISDataset(x_test, y_test)

train_loader = DataLoader(iris_dataset_train, batch_size=8, shuffle=True)
test_loader = DataLoader(iris_dataset_test, batch_size=8, shuffle=True)

for x, y in train_loader:
  print(x, y)

# _**Multi Layer Perceptron**_

class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.l1 = nn.Linear(4, 10)
    self.l2 = nn.Linear(10, 12)
    self.l3 = nn.Linear(12, 3)
  
  def forward(self, x):
    x1 = self.l1(x)
    x1_tanh = torch.tanh(x1)
    x2 = self.l2(x1_tanh)
    x2_sigmoid = torch.sigmoid(x2)
    x3 = self.l3(x2_sigmoid)
    out = torch.softmax(x3, dim=1)
    return out

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.1)

# _**Training Loop**_

for epoch in range(80):
  epoch_loss = []
  epoch_acc = []
  for x, y in train_loader:
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()
    acc_score = accuracy_score(y_true=y, y_pred=pred.argmax(dim=1))

    epoch_loss.append(loss.item())
    epoch_acc.append(acc_score)

  print(f"epoch: {epoch}, loss: {np.mean(epoch_loss)}, accuracy: {np.mean(epoch_acc)}")


with torch.no_grad():
  epoch_loss = []
  epoch_acc = []
  for x, y in test_loader:
    pred = model(x)
    loss = criterion(pred, y)
    epoch_loss.append(loss.item())

    acc = accuracy_score(pred.argmax(dim=1), y)
    epoch_acc.append(acc)

    print(f"test loss: {np.mean(epoch_loss)}, test accuracy: {np.mean(epoch_acc)}")

